<a href="https://colab.research.google.com/github/bkaplowitz/Tax-Smoothing/blob/master/tax_smoothing_examples.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Examples for Brandon

Here are some examples of Markov chains to put into the code you revised yesterday to accommodate $N > 2$

In the following examples the parameteres $\lambda, \gamma, \phi$ are all fractions living in $[0,1]$

You can set them all to $.5$ initially, but we can have fun experimenting with other values

Please make the output "verbose" at this stage so that you print out lots of the objects that the code computes,
e.g., debt levels, tax  collections, and so on.

In [2]:
!pip install --upgrade quantecon

     |████████████████████████████████| 143kB 2.8MB/s 
  Created wheel for quantecon: filename=quantecon-0.4.5-cp36-none-any.whl size=166733 sha256=216d607e1ce52b4d6a532b6a3b2812298d53805266467b9836ac7043ff650877
  Stored in directory: /root/.cache/pip/wheels/cf/80/42/ea1feb22ce8fd1f2f230bcce9c104c490fa376a01c8e9faf79
Successfully built quantecon


In [1]:
import numpy as np
import quantecon as qe
import scipy.linalg as la


def isprobabilitymatrix(P: np.ndarray):
  try:
    assert(np.all(P>=0)==True)
  except AssertionError:
    print("Matrix has negative entries")
  k,l=P.shape
  try:
    assert(k==l)
  except AssertionError:
    print("Matrix is non-symmetric")
  try:
    assert(np.all(np.sum(P,axis=1,keepdims=True)==np.ones((1,k))))
  except AssertionError:
    print("Matrix rows do not sum to 1")
  try:
    assert(isinstance(P,np.ndarray))
  except AssertionError:
    print("Not array-like")

class ConsumptionProblem:
    """
    The data for a consumption problem, including some default values.
    """

    def __init__(self,
                 β=.96,
                 y=[2, 1.5,3],
                 b0=3,
                 P=np.asarray([[.8, .1, .1],
                               [.2, .6,.2],
                               [.3,.4,.3]])):
        """

        Parameters
        ----------

        β : discount factor
        P : n x n transition matrix
        y : list containing the n income levels
        b0 : debt in period 0 (= state_1 debt level)

        """
        try:
          assert((β>=0) & (β<=1))
        except AssertionError:
          print('β must be in (0,1)')
        try:
          assert(np.all(np.isreal(y))==True)
        except AssertionError:
          print('y must be a real number')
        try:
           assert(np.all(np.isreal(b0))==True)
        except AssertionError:
          print('b0 must be a real number')
        isprobabilitymatrix(P) 
        self.β = β
        self.y = y
        self.b0 = b0
        self.P = P
        self.n = np.size(y)

  
      
def consumption_complete(cp):
    """
    Computes endogenous values for the complete market case.

    Parameters
    ----------

    cp : instance of ConsumptionProblem

    Returns
    -------

        c_bar : constant consumption
        b: vector of debts in each state

    associated with the price system 

        Q = β * P

    """

    β, P, y, b0, n = cp.β, cp.P, cp.y, cp.b0, cp.n   # Unpack
    #b[0] = b0                               # b(s1) is known to be equal to b0
    Q = β * P                               # assumed price system
    
    # Using equation (7-9) calculate b2, where we will stack the time 0 equation
    # on top of our existing n equations. Our new y variable will represent all 
    # net income, with the top being y(s_1)-b_0
    # our new b variable will be called X and will capture all choice variables
    # with the first entry being c_bar.
    # X= [c_bar; b]
    # G  [1 0 0 0 0 0...; 
    #     1 1 0........;
    #     1 0 1 ...]
    #E.g [ 1, 0 ]
    #    [ 1, I ]
    #G@X = [y(s_1)-b_0;Y_VEC]+Q_new (left entry is all 0's now)@np.asarray(X) 
    #c_bar = y(s_1)+Q[0,:]@np.asarray(b)-b_0
    # (G-Q_new)@X=[y(s_1)-b_0;Y_VEC]
    # X= (G-Q_new)^{-1}@[y(s_1)-b_0;Y_VEC]
    G = np.block([ #represents the fact that we don't observe b and c_bar seperately, only b(s_j)+c_bar on LHS
                  [1,np.zeros((1,n))],
                  [np.ones((n,1)), np.eye(n)]
                  ])
    y_aug= np.concatenate((np.atleast_2d(y[0]-b0),np.atleast_2d(y).T)) #construct our new y, state variables
    Q_aug = np.block([ #construct our new Q matrix that involves both the time 0 prices (same as state 1) and prices for all other states
                  [0,Q[0,:]],
                  [np.zeros((n,1)),Q]                
    ])
    X = np.linalg.inv((G-Q_aug)) @ y_aug # solve for X using the linear algebra calculations above.
    c_bar=X[0,:] #unroll solved for x
    b = X[1:n+1,:]
    c_bar = np.ndarray.item(c_bar)
    b= b.flatten()
    return c_bar, b


def consumption_incomplete(cp, N_simul=150):
    """
    Computes endogenous values for the incomplete market case.

    Parameters
    ----------

    cp : instance of ConsumptionProblem
    N_simul : int

    """

    β, P, y, b0 = cp.β, cp.P, cp.y, cp.b0  # Unpack
    # For the simulation define a quantecon MC class
    mc = qe.MarkovChain(P)

    # Useful variables
    y = np.asarray(y).reshape(n, 1)
    v = np.linalg.inv(np.eye(n) - β * P) @ y

    # Simulat state path
    s_path = mc.simulate(N_simul, init=0)

    # Store consumption and debt path
    b_path, c_path = np.ones(N_simul + 1), np.ones(N_simul)
    b_path[0] = b0

    # Optimal decisions from (12) and (13)
    db = ((1 - β) * v - y) / β

    for i, s in enumerate(s_path):
        c_path[i] = (1 - β) * (v - b_path[i] * np.ones((np.size(y), 1)))[s, 0]
        b_path[i + 1] = b_path[i] + db[s, 0]

    return c_path, b_path[:-1], y[s_path], s_path

ModuleNotFoundError: ignored

### Example 1

This example is designed to emulate the Civil War and WWI

$$ P = \begin{bmatrix} 1 - \lambda & \lambda & 0 \cr
                       0 & 1 - \phi & \phi \cr
                       0 & 0 & 1 \end{bmatrix} $$

where the government expenditure vector  $g = \begin{bmatrix} g_L & g_H & g_M \end{bmatrix}$
where $ g_L < g_M < g_H$ .  Please assume $b_0 = 0$ and assume that the initial Markov state is
state $1$ so that the system starts off in peace.

Please apply the code for both the complete and the incomplete markets tax-smoothing models and plot
outcomes as they are in one of the graphs in the present notebook from quantecon or from your notebook from
yesterday

### Example 2

This example captures a permanent peace.  

Here set

$$ P = \begin{bmatrix}   1 & 0 & 0 \cr
                         0 & 1-\gamma & \gamma \cr
                         \phi & 0 & 1-\phi \end{bmatrix} $$
                       
where the government expenditure vector  $g = \begin{bmatrix} g_L & g_L & g_H \end{bmatrix}$
where $ g_L < g_H$ .  Please assume $b_0 = 0$ and assume that the initial Markov state is state $2$ so that 
the system starts off in a temporary peace.  


Please apply the code for both the complete and the incomplete markets tax-smoothing models and plot
outcomes as they are in one of the graphs in the present notebook from quantecon or from your notebook from
yesterday




state $1$ so that the system starts off in peace.          